# Lab-7: ANN in Keras

In this lab, you will practice simple deep learning model in Pytorch.


## Objectives:
1. Theoretical issues
2. Get starting in Pytorch
___

### Theoretical issues
Ordinary fully connected neural nets consists of Dense layers, activations, and output layer.

1. What's the difference between deep learning and normal machine learning?
2. How does a neural network with no hidden layers and one output neuron compare to a logistic/linear regression?
3. How does a neural network with multiple hidden layers but with linear activation and one output neuron compared to logistic/linear regression?
4. Can the perceptron find a non-linear decision boundary?
5. In multi-hidden layers network, what's the need of non-linear activation function?
6. Is random weight assignment better than assigning same weights to the units in the hidden layer.
---

### Pytorch: Getting started
### Feed Forward Neural Network
an artificial neural network wherein connections between the nodes do not form a cycle.
![alt text](https://upload.wikimedia.org/wikipedia/en/5/54/Feed_forward_neural_net.gif)

## Model Desing in Pytorch
we have three simple parts that we need to build:
1. Data Loading process.
2. Model building.
3. the training loops.

### 1. Data Loading

Data Loading in pytorch is very easy and broken into 3 steps:
1. Data Source.
2. Data Transformations.
3. Data Loader.



#### Loading data

Pytorch uses data loading utility which is called `DataLoader` that supports:
automatic batching, transformation, single- and multi-process data loading and more..

In [1]:
from torchvision import datasets, transforms
from torch. utils.data import DataLoader

batch_size = 32
test_batch_size = 100

data_transformations = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])

mnist_train = datasets.MNIST('../data', train=True, download=True,
                       transform=data_transformations)
mnist_test = datasets.MNIST('../data', train=False,
                            transform=data_transformations)

train_loader = DataLoader(mnist_train,
                          batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test,
                         batch_size=test_batch_size, shuffle=True)

0it [00:00, ?it/s]Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to ../data/MNIST/raw/train-images-idx3-ubyte.gz
9920512it [00:02, 3995211.77it/s]
Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw
0it [00:00, ?it/s]Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz to ../data/MNIST/raw/train-labels-idx1-ubyte.gz
32768it [00:00, 76579.01it/s]
0it [00:00, ?it/s]Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw
1654784it [00:01, 1493430.11it/s]
0it [00:00, ?it/s]Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw
8192it [00:00, 23726.10it/s]Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!



#### Model building
1. Defining components: <br/>
This step is done in the constructor, where you will define the layers that will be used accordingly in the next step.
2. Network flow: <br/>
This step is done in the forward function. Where you will get the input batch as an argument then you will use the defined layers in the previous step to define the flow of the network then you will return the output batch.


Pytorch is a dynamic framework, where you can use primitive python keywords with it.
You can use if and while statements. Also, it can accepts and returns more than one batch.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)

        # Write 3 lines to define 3 more linear layers.
        # 2 hidden layers with number of neurons numbers: 250 and 100
        # 1 output layer that should output 10 neurons, one for each class.
        self.fc2 = nn.Linear(500, 250)
        self.fc3 = nn.Linear(250, 100)
        self.fc4 = nn.Linear(100, 10)



    def forward(self, x):
        # the linear layers fc1, fc2, fc3, and fc4
        # accepts only flattened input (1D batches)
        # while the batch x is of size (batch, 28 * 28)
        # define one line to flatten the x to be of size (batch_sz, 28 * 28)
        x = x.view(-1, 28*28)
        # x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)

### Training loops
After that we should define the loops over tha batches and run the training on.

In [4]:
epochs = 10
lr = 0.01
momentum = 0.5
seed = 1
log_interval = 10

In [5]:
def train( model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

In [3]:
def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # Do the same that was done in the previous function.
            # But without backprobagating the loss and without running the optimizers
            # As this function is only for test.
            # write 3 lines to transform the data to the device, get the output and compute the loss
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.nll_loss(output, target, reduction='sum')
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)


for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

torch.save(model.state_dict(), "mnist_model.pt")



 8 [54080/60000 (90%)]	Loss: 0.011104
Train Epoch: 8 [54400/60000 (91%)]	Loss: 0.004353
Train Epoch: 8 [54720/60000 (91%)]	Loss: 0.003895
Train Epoch: 8 [55040/60000 (92%)]	Loss: 0.016183
Train Epoch: 8 [55360/60000 (92%)]	Loss: 0.005486
Train Epoch: 8 [55680/60000 (93%)]	Loss: 0.017044
Train Epoch: 8 [56000/60000 (93%)]	Loss: 0.037585
Train Epoch: 8 [56320/60000 (94%)]	Loss: 0.006515
Train Epoch: 8 [56640/60000 (94%)]	Loss: 0.013211
Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.008241
Train Epoch: 8 [57280/60000 (95%)]	Loss: 0.013951
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.032452
Train Epoch: 8 [57920/60000 (97%)]	Loss: 0.020794
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.037458
Train Epoch: 8 [58560/60000 (98%)]	Loss: 0.011813
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.005958
Train Epoch: 8 [59200/60000 (99%)]	Loss: 0.010265
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.096516
Train Epoch: 8 [59840/60000 (100%)]	Loss: 0.077861

Test set: Average loss: 0.0000, Accuracy: 9773/10000 (98%)

